# Notebook 3: Memory Integration

This notebook demonstrates:
1. LangGraph memory integration
2. Conversational RAG sessions
3. Multi-turn dialogues with context
4. Chat history management

In [ ]:
import sys
sys.path.append('..')

from src.memory import (
    ConversationMemory,
    create_memory_enabled_rag,
    ChatSession,
    create_langgraph_memory
)
from src.rag_baseline import BaselineRAG
from dotenv import load_dotenv
import os

load_dotenv()
print("✓ Imports successful")

## Step 1: Basic Conversation Memory

In [ ]:
# Test basic memory
memory = ConversationMemory(max_messages=10)

memory.add_message("user", "What is GDPR?")
memory.add_message("assistant", "GDPR is...")
memory.add_message("user", "Tell me more about that")

print("Conversation History:")
print(memory.get_context_string())

## Step 2: Memory-Enabled RAG

In [ ]:
# Create RAG with memory
base_rag = BaselineRAG("../faiss_index", os.getenv("OPENAI_API_KEY"))
memory_rag = create_memory_enabled_rag(base_rag)

# Multi-turn conversation
print("Turn 1:")
answer1 = memory_rag.query("What are the key principles of GDPR?")
print(answer1)

print("\nTurn 2:")
answer2 = memory_rag.query("Can you explain the first one in detail?")
print(answer2)

## Step 3: Chat Session

In [ ]:
# Create chat session
session = ChatSession(base_rag, session_id="demo-session")

# Simulate conversation
queries = [
    "What is GDPR?",
    "What are the penalties?",
    "Who does it apply to?"
]

for query in queries:
    response = session.chat(query)
    print(f"\nUser: {query}")
    print(f"Assistant: {response[:200]}...")

print(f"\nTotal turns: {session.turn_count}")

## Step 4: LangGraph Memory (Advanced)

In [ ]:
# Create LangGraph memory configuration
langgraph_memory = create_langgraph_memory(checkpointer_path="../checkpoints")

print("LangGraph memory configuration:")
print(langgraph_memory)

## Summary

✓ Implemented conversation memory
✓ Created memory-enabled RAG
✓ Tested multi-turn dialogues
✓ Explored LangGraph integration

Next: `04_guardrails.ipynb` for safety filters